# Exemplo 

## Precificação de imóveis

* Como utilizar o *PyCaret* para a precificação de imóveis. Assim, com base num conjunto de preditores busca-se estimar o preço de uma residência.

### Estratégias de negócio

* Utilizar a estimativa do preço do imóvel para precificá-lo ao consumidor final.
* Priorizar a venda de imóveis dependendo da estimativa do preço.
* Alocar o corretor para a venda do imóvel com base no preço estimado.

**Imports requeridos para a execução do estudo**

In [1]:
#Bibliotecas básicas
import pandas as pd     #Manipulação dos dados
import numpy as np      #Operações multidimensionais e matemáticas
import matplotlib.pyplot as plt    #Gráficos
import matplotlib.ticker as ticker #Remover a notação científica do gráfico
import seaborn as sns              #Gráficos
##Machine learning
from pycaret.regression import * 
##Eliminar os warnings
import warnings
warnings.filterwarnings("ignore")
##Ver todas as colunas do data frame
pd.set_option('display.max_columns', None)
##Extrair os valores do feature importance
import sklearn as sk
#Definir o formato de exibição tipo float para evitar notação científica
pd.options.display.float_format = '{:.2f}'.format
#MAPE (métrica de avaliação)
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
##Extrair os valores do feature importance
import sklearn as sk

**Versões do Python e Pycaret instalados**

In [2]:
#Identificar a versão do Python
import sys
print(f"Versão do Python: {sys.version}") #Versão do Python: 3.11.8

Versão do Python: 3.11.8 | packaged by Anaconda, Inc. | (main, Feb 26 2024, 21:34:05) [MSC v.1916 64 bit (AMD64)]


In [3]:
#Identificar a versão do Pycaret
import pycaret
print(f"Versão do PyCaret: {pycaret.__version__}") #Versão do PyCaret: 3.3.0

Versão do PyCaret: 3.3.0


# 1 - Visão geral os dados

In [4]:
#Importar a base de dados
df =  pd.read_csv('base_dados.csv', sep = ';')
#Visualizar
df.head(3)

,ID_RESIDENCIA,PRECO,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA
0,A1,13300000,7420,4.00,2.00,3,SIM,NAO,NAO,NaN,SIM,2.00,SIM,SIM
1,A2,12250000,8960,4.00,4.00,4,SIM,NAO,NaN,NAO,SIM,3.00,NAO,SIM
2,A3,12250000,9960,NaN,2.00,2,SIM,NAO,SIM,NAO,NAO,2.00,SIM,PARCIALMENTE


* NaN - **not as number** (é um valor nulo/ *missing*).

In [5]:
#Volumetria 
df.shape[0] #Total de observações: 545

545

## Observações iniciais

In [6]:
#Características da base de dados
df.info()#Nome da variável, contagem de observações não nulas e o tipo das variáveis (object - string/texto, int64 - número inteiro, float64 - decimal)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_RESIDENCIA           545 non-null    object 
 1   PRECO                   545 non-null    int64  
 2   AREA                    545 non-null    int64  
 3   QUARTOS                 481 non-null    float64
 4   BANHEIROS               465 non-null    float64
 5   ANDARES                 545 non-null    int64  
 6   FLAG_CENTRO             533 non-null    object 
 7   FLAG_QUARTO_HOSPEDE     523 non-null    object 
 8   FLAG_PORAO              510 non-null    object 
 9   FLAG_AGUA_MORNA         469 non-null    object 
 10  FLAG_AR_CONDICIONADO    532 non-null    object 
 11  VAGAS_ESTACIONAMENTO    485 non-null    float64
 12  FLAG_AREA_PREFERENCIAL  527 non-null    object 
 13  MOBILIADA               493 non-null    object 
dtypes: float64(3), int64(3), object(8)
memory 

In [7]:
#ID_RESIDENCIA - é o código de casa
##Espera-se que cada linha apresente um único código
###Conatgem única de uma variável (elementos únicos que esta variável tem)
df['ID_RESIDENCIA'].drop_duplicates().shape[0] #545 - Cada linha é um único ID 

545

**Observações sobre variáveis categóricas**:

* Variável Categórica Ordinal: Tem ordem implícita.
    * **Exemplo**: *Nível de Escolaridade* (Valores: Fundamental < Médio < Superior).
* Variável Categórica Nominal:  Sem ordem natural.
    * **Exemplo**: *Cor dos Olhos* (Valores: Azul, Verde, Castanho).

In [8]:
#Quais as classes da variável MOBILIADA
df['MOBILIADA'].drop_duplicates() #Essa variável pode ser vista como ordinal (ordem/ ranqueamento)
                                  #Uma casa mobiliada pode ser vista como uma casa melhor que uma parcial ou sem mobília

0              SIM
2     PARCIALMENTE
7              NAO
12             NaN
Name: MOBILIADA, dtype: object

In [9]:
#Frequência de uma variável categórica (absoluta ou relativa)
df['MOBILIADA'].value_counts(True) #Pergunta -- que tipo de análise eu poderia fazer para as variáveis quantitativas?

PARCIALMENTE   0.42
NAO            0.31
SIM            0.26
Name: MOBILIADA, dtype: float64

# 2 - PyCaret

## 2.1 - Setup

* Esta função tem vários parâmetros e prepara o ambiente de modelagem, ou seja, realiza o pré-processamento dos dados (**imputação**, **normalização** e **encoding**) e os divide entre treinamento e teste.

In [10]:
exp = setup(df, target = 'PRECO', session_id = 1935, train_size = 0.65, ignore_features=['ID_RESIDENCIA'],
            normalize = True, normalize_method = 'minmax', numeric_imputation = 'median', categorical_imputation = 'mode',
            ordinal_features = {'MOBILIADA': ['NAO','PARCIALMENTE', 'SIM'],
                               'FLAG_CENTRO': ['NAO', 'SIM'],
                               'FLAG_QUARTO_HOSPEDE': ['NAO', 'SIM'],
                               'FLAG_PORAO': ['NAO', 'SIM'],
                               'FLAG_AGUA_MORNA': ['NAO', 'SIM'],
                               'FLAG_AR_CONDICIONADO': ['NAO', 'SIM'],
                               'FLAG_AREA_PREFERENCIAL': ['NAO', 'SIM']}, experiment_name= "EXP_REGRESSAO")

,Description,Value
0,Session id,1935
1,Target,PRECO
2,Target type,Regression
3,Original data shape,"(545, 14)"
4,Transformed data shape,"(545, 15)"
5,Transformed train set shape,"(354, 15)"
6,Transformed test set shape,"(191, 15)"
7,Ignore features,1
8,Ordinal features,7
9,Numeric features,5


In [11]:
#Base de treino
df_treino = get_config('train')
#Visualização
df_treino.head(3)

,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA,PRECO
229,9667,4.00,2.00,2,SIM,SIM,SIM,NAO,NAO,1.00,NAO,PARCIALMENTE,4690000
114,6800,NaN,1.00,1,SIM,SIM,SIM,NAO,NAO,NaN,NAO,SIM,6020000
405,3060,3.00,1.00,1,SIM,NAO,NAO,NAO,NAO,0.00,NaN,NAO,3465000


In [12]:
#As variáveis da base de treino transformadas após o pré-processamento
X = get_config('X_train_transformed')
X.head(3)

,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA_1.0,MOBILIADA_2.0,MOBILIADA_0.0
229,0.55,0.60,0.33,0.33,1.00,1.00,1.00,0.00,0.00,0.33,0.00,1.00,0.00,0.00
114,0.35,0.40,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
405,0.10,0.40,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00


In [13]:
#Volumetria da base de treino
df_treino.shape[0] #354

354

### Comparando a variável AREA - original vs transformada

* **Original**

In [14]:
df_treino['AREA'].describe()

count     354.00
mean     5216.04
std      2245.19
min      1650.00
25%      3600.00
50%      4600.00
75%      6420.00
max     16200.00
Name: AREA, dtype: float64

* **Transformada**

In [15]:
X['AREA'].describe()

count   354.00
mean      0.25
std       0.15
min       0.00
25%       0.13
50%       0.20
75%       0.33
max       1.00
Name: AREA, dtype: float64

### Base de teste

In [16]:
#Base de teste - usada para a avaliação final dos modelos
df_teste = get_config('test')
#Visualização
df_teste.head(3)

,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA,PRECO
483,6615,3.00,NaN,2,SIM,NAO,NAO,NaN,NAO,0.00,NAO,PARCIALMENTE,2940000
172,8400,NaN,1.00,2,SIM,SIM,SIM,NAO,SIM,2.00,SIM,NAO,5250000
144,4700,4.00,1.00,2,SIM,SIM,SIM,NAO,SIM,1.00,NAO,SIM,5600000


## 2.2 - Compare models

* Esta função treina e avalia o desempenho de todos os algoritmos disponíveis através da abordagem do cross-validation (o padrão são 10 folds). Ela fornece um resumo das métricas de avaliação usadas para cada modelo.

In [17]:
#Quais algoritmos estão disponíveis (instalados)
models() #Todos os que possuem True estão instalados, podendo ser testados. Se for False, é possível instalar o algoritmo e utilizá-lo.

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


### 2.2.1 - Métrica de avaliação: MAPE (Erro Médio Percentual Absoluto)



#### Explicação 

* o MAPE indica, em média, o quão distantes estão as previsões dos valores reais, em termos percentuais. **Quanto menor o valor do MAPE, mais precisa é a previsão**. Por exemplo, um MAPE de 5% indica que, em média, as previsões estão, em média, a 5% dos valores reais (seja para mais ou menos).


$$
\text{MAPE} = \frac{1}{n} \sum_{i=1}^{n} \left( \frac{|A_i - F_i|}{A_i} \right) \times 100
$$

##### Notação

* A é o valor real;
* F é o valor previsto;
* n é o número total de observações na amostra,
* Σ representa a soma sobre todas as observações,
* | | representa o valor absoluto.
    

In [23]:
#Ordenar os modelos com base nos que tem o menor MAPE - métrica de avaliação adotada
compare_models(sort = 'MAPE') 
#Ordenar os modelos do menor para o maior MAPE

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,798444.9154,1170970875337.2876,1064002.9883,0.6434,0.2146,0.1773,0.0720
lasso,Lasso Regression,801914.9289,1174788535375.5710,1066425.4273,0.6411,0.2155,0.1777,0.0800
lar,Least Angle Regression,801915.5597,1174787963312.4150,1066425.3492,0.6411,0.2155,0.1777,0.0710
llar,Lasso Least Angle Regression,801914.9124,1174788451600.9600,1066425.3740,0.6411,0.2155,0.1777,0.0680
lr,Linear Regression,801439.1203,1171845642690.0366,1065771.5351,0.6415,0.2157,0.1778,0.0860
rf,Random Forest Regressor,825826.9176,1309991595265.3264,1119201.9368,0.6056,0.2228,0.1817,0.1960
catboost,CatBoost Regressor,832400.1586,1354656084645.7449,1133500.6880,0.5983,0.2238,0.1820,0.8890
lightgbm,Light Gradient Boosting Machine,838711.6148,1393826424273.3423,1149161.9948,0.5745,0.2237,0.1825,0.1930
gbr,Gradient Boosting Regressor,872147.4795,1466555121410.4265,1178926.1253,0.5597,0.2315,0.1911,0.1310
xgboost,Extreme Gradient Boosting,888222.2829,1577265282402.2122,1226937.4154,0.5310,0.2422,0.1924,0.1110


Ridge(random_state=1935)

## 2.3 - Create model

* Esta função treina e avalia o desempenho de um determinado estimador utilizando a abordagem do cross-validation.

# Regressão Ridge  

A **Regressão Ridge** é uma variação da regressão linear que adiciona uma penalização **L2** (soma dos quadrados dos coeficientes) para evitar *overfitting* e melhorar a generalização do modelo.  

## Parâmetro Principal: `alpha` (λ)  

### O que faz?  
Controla a intensidade da regularização:  

- **`alpha = 0`** → Desativa a penalização (igual à regressão linear simples).  
- **`alpha > 0`** → Reduz a magnitude dos coeficientes, evitando valores extremos.  
- **Quanto maior o `alpha`**, mais os coeficientes se aproximam de zero (sem nunca zerar).  

In [25]:
#Modelo Ridge
model_rigde = create_model('ridge') 
model_rigde  #Na base de treino se aplica o cross validação

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,756037.4394,998458272750.5084,999228.8390,0.6927,0.2025,0.1629
1,846974.6177,1386433224129.9543,1177468.9907,0.4740,0.2571,0.2085
2,792471.3935,1098912921885.7528,1048290.4759,0.6009,0.2319,0.1975
3,766584.6035,1069095800891.7437,1033970.8898,0.7175,0.2003,0.1632
4,800426.5574,1152688897098.6707,1073633.5022,0.7677,0.2231,0.1861
5,910730.5534,1172717630443.9841,1082920.8791,0.5720,0.2457,0.2241
6,658126.0667,729652095867.0696,854196.7548,0.6871,0.1710,0.1361
7,1024008.5204,2414841560050.4360,1553976.0487,0.5937,0.2369,0.1838
8,785501.9918,1091690895918.2769,1044840.1294,0.5857,0.2229,0.1732


Ridge(random_state=1935)

In [29]:
print(model_rigde.alpha) #alpha = 1

1.0


In [26]:
#Avaliar o modelo na base de teste
##Criar um data frame
df_teste_predicao = predict_model(model_rigde, data = df_teste)
#Visualização
df_teste_predicao.head(3)
#MAPE = 0.1876

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Ridge Regression,853068.9396,1468199608915.9065,1211692.8691,0.5940,0.2337,0.1876


,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA,PRECO,prediction_label
483,6615,3.00,NaN,2,SIM,NAO,NAO,NaN,NAO,0.00,NAO,PARCIALMENTE,2940000,4175094.22
172,8400,NaN,1.00,2,SIM,SIM,SIM,NAO,SIM,2.00,SIM,NAO,5250000,7533131.50
144,4700,4.00,1.00,2,SIM,SIM,SIM,NAO,SIM,1.00,NAO,SIM,5600000,6310951.16


In [27]:
#Outra abordagem de extrair o MAPE da base de teste
mape_1 = mean_absolute_percentage_error(df_teste_predicao.PRECO, df_teste_predicao.prediction_label)
print("MAPE:", round(100 * mape_1,2)) #MAPE: 18.76

MAPE: 18.76


## 2.4 - Tune model (opcional)

* Esta função visa encontrar uma nova combinação de hiperparâmetros que possa melhorar a performance do modelo.

In [40]:
#Simular outros valores de alfa para encontrar um resultado melhor
param = {'alpha': [0.0001, 0.01, 0.1, 1, 2, 5, 10]}

In [41]:
tuned_model = tune_model(model_rigde, optimize = 'MAPE', fold = 10, 
                        custom_grid = param, n_iter=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,756037.4394,998458272750.5084,999228.8390,0.6927,0.2025,0.1629
1,846974.6177,1386433224129.9543,1177468.9907,0.4740,0.2571,0.2085
2,792471.3935,1098912921885.7528,1048290.4759,0.6009,0.2319,0.1975
3,766584.6035,1069095800891.7437,1033970.8898,0.7175,0.2003,0.1632
4,800426.5574,1152688897098.6707,1073633.5022,0.7677,0.2231,0.1861
5,910730.5534,1172717630443.9841,1082920.8791,0.5720,0.2457,0.2241
6,658126.0667,729652095867.0696,854196.7548,0.6871,0.1710,0.1361
7,1024008.5204,2414841560050.4360,1553976.0487,0.5937,0.2369,0.1838
8,785501.9918,1091690895918.2769,1044840.1294,0.5857,0.2229,0.1732


Fitting 10 folds for each of 7 candidates, totalling 70 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [42]:
#Avaliar o modelo tunado na base de teste
df_teste_tune = predict_model(tuned_model, data = df_teste)
#Visualização
df_teste_tune.head(3) #MAPE: 0.1876

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Ridge Regression,853068.9396,1468199608915.9065,1211692.8691,0.5940,0.2337,0.1876


,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA,PRECO,prediction_label
483,6615,3.00,NaN,2,SIM,NAO,NAO,NaN,NAO,0.00,NAO,PARCIALMENTE,2940000,4175094.22
172,8400,NaN,1.00,2,SIM,SIM,SIM,NAO,SIM,2.00,SIM,NAO,5250000,7533131.50
144,4700,4.00,1.00,2,SIM,SIM,SIM,NAO,SIM,1.00,NAO,SIM,5600000,6310951.16


In [43]:
#valor do alpha = 1
print(tuned_model.alpha)

1.0


## 2.5 - Avaliação de desempenho

In [44]:
#Panorama geral dos resultados
evaluate_model(model_rigde)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [45]:
#Top 10 variáveis
plot_model(model_rigde, plot = 'feature', save=True)

'Feature Importance.png'

In [46]:
#Ordem de importância de todas as variáveis
plot_model(model_rigde, plot = 'feature_all', save=True)

'Feature Importance (All).png'

In [47]:
#Criar um data frame do feature importance (o peso dos coeficientes estimados)
feature_importance = model_rigde.coef_
feature_names = X.columns
feature_importance_dict = dict(zip(feature_names, feature_importance))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
df_importance = pd.DataFrame(sorted_feature_importance, columns=['Feature', 'Importance'])
df_importance.to_csv("feature_importance.csv", sep=";")
df_importance 

,Feature,Importance
0,AREA,2966935.35
1,BANHEIROS,2181051.05
2,ANDARES,1766142.39
3,FLAG_AGUA_MORNA,920712.36
4,VAGAS_ESTACIONAMENTO,887097.47
5,FLAG_AR_CONDICIONADO,831381.89
6,FLAG_AREA_PREFERENCIAL,685372.06
7,FLAG_QUARTO_HOSPEDE,601221.15
8,FLAG_PORAO,487605.95
9,QUARTOS,422934.13


In [48]:
#Intercepto estimado
tuned_model.intercept_ #2049016.8354677442

2049016.8354677442

In [50]:
#Salvar o erro
plot_model(model_rigde, plot = 'error', save=True)

'Prediction Error.png'

In [51]:
#Salvar os resíduos
plot_model(model_rigde, plot = 'residuals', save=True)

'Residuals.png'

In [52]:
#Pipeline da modelagem
plot_model(model_rigde, plot = 'pipeline', save=True)

'Pipeline Plot.png'

## 2.6 - Concluir o experimento

In [53]:
#Finalizar o modelo
##O modelo vai ser treinado com todo o data set (base completa), considerando a regressão lasso.
final_model = finalize_model(model_rigde)
final_model

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['AREA', 'QUARTOS', 'BANHEIROS',
                                             'ANDARES',
                                             'VAGAS_ESTACIONAMENTO'],
                                    transformer=SimpleImputer(strategy='median'))),
                ('categorical_imputer',
                 TransformerWrapper(include=['FLAG_CENTRO',
                                             'FLAG_QUARTO_HOSPEDE',
                                             'FLAG_PORAO', 'FLAG_AGUA_MORNA',
                                             'FLAG_AR_CONDICIONADO',
                                             'FLAG...
                                                                        {'col': 'MOBILIADA',
                                                                         'data_type': dtype('O'),
                                                                         'mapping': NAO             0
PARCIALMENTE    1
SIM             2
NaN            -1
dtype: int64}]))),
                ('onehot_encoding',
                 TransformerWrapper(include=['MOBILIADA'],
                                    transformer=OneHotEncoder(cols=['MOBILIADA'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('normalize', TransformerWrapper(transformer=MinMaxScaler())),
                ('actual_estimator', Ridge(random_state=1935))])

In [54]:
previsoes_df = predict_model(final_model, data=df)
#Visualização
previsoes_df.head(3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Ridge Regression,784614.3888,1164816342145.1650,1079266.5760,0.6664,0.2153,0.1739


,ID_RESIDENCIA,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA,PRECO,prediction_label
0,A1,7420,4.00,2.00,3,SIM,NAO,NAO,NaN,SIM,2.00,SIM,SIM,13300000,8239445.32
1,A2,8960,4.00,4.00,4,SIM,NAO,NaN,NAO,SIM,3.00,NAO,SIM,12250000,10359804.79
2,A3,9960,NaN,2.00,2,SIM,NAO,SIM,NAO,NAO,2.00,SIM,PARCIALMENTE,12250000,7553090.18


In [55]:
#Remomear a variável prediction_label
previsoes_df.rename(columns = {'prediction_label':'PRECO_ESTIMADO'}, inplace = True)

In [56]:
#Visualizar
previsoes_df.head(3)

,ID_RESIDENCIA,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA,PRECO,PRECO_ESTIMADO
0,A1,7420,4.00,2.00,3,SIM,NAO,NAO,NaN,SIM,2.00,SIM,SIM,13300000,8239445.32
1,A2,8960,4.00,4.00,4,SIM,NAO,NaN,NAO,SIM,3.00,NAO,SIM,12250000,10359804.79
2,A3,9960,NaN,2.00,2,SIM,NAO,SIM,NAO,NAO,2.00,SIM,PARCIALMENTE,12250000,7553090.18


In [57]:
#Salvar o data frame
previsoes_df.to_csv('df_estimativas.csv', header=True, index=False, sep=';')

In [58]:
#Salvar o modelo ("CPF" do modelo)
save_model(final_model, 'rigde_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['AREA', 'QUARTOS', 'BANHEIROS',
                                              'ANDARES',
                                              'VAGAS_ESTACIONAMENTO'],
                                     transformer=SimpleImputer(strategy='median'))),
                 ('categorical_imputer',
                  TransformerWrapper(include=['FLAG_CENTRO',
                                              'FLAG_QUARTO_HOSPEDE',
                                              'FLAG_PORAO', 'FLAG_AGUA_MORNA',
                                              'FLAG_AR_CONDICIONADO',
                                              'FLAG...
                                                                         {'col': 'MOBILIADA',
                                                                          'data_type': dtype('O'),
                                                              